This notebook is part of Andreu's (esdandreu@gmail.com) Master Thesis work at
Keio University.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AcousticOdometry/AO/blob/main/notebooks/plot_features.ipynb)


# Setup
This section will take care of installing the necessary packages as well as
configuring some environment variables.

In [3]:
COLAB_RUNTIME = False
GITHUB_TOKEN = None

## Colab
Assess wether the notebook is being executed in [Google
Colab](https://colab.research.google.com/) and if so, set up the software
needed in Colab runtime.

In [4]:
%%capture
try:
    import colab
    COLAB_RUNTIME = True
    # Check CMake >= 3.21
    v_str, *_ = !cmake --version
    if 'command not found' in v_str:
        major, minor = (0, 0)
    else:
        major, minor, _ = (int(x) for x in v_str.split(' ')[-1].split('.'))
    if major < 3 or minor < 21:
        # https://cmake.org/download/
        %cd /tmp
        !wget https://github.com/Kitware/CMake/releases/download/v3.22.3/cmake-3.22.3-linux-x86_64.sh
        !sudo mkdir /opt/cmake
        !sudo sh ./cmake-3.22.3-linux-x86_64.sh --prefix=/opt/cmake \
            --skip-license
        %cd /content
        !update-alternatives --install /usr/local/bin/cmake cmake \
            /opt/cmake/bin/cmake 20 --force
except ImportError:
    pass

## AO
Setup Acoustic Odometry python package. If this notebook is being executed in
[Colab](#colab), the package will be installed from Github. Because of this, a
Github [personal access
token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token)
will be asked during the installation.

If the notebook is not running on Colab and the package is not already
installed, installation instructions will be prompted.

In [5]:
if COLAB_RUNTIME:
    from getpass import getpass
    if not GITHUB_TOKEN:
        GITHUB_TOKEN = getpass(
            "Personal access token\n"
            r"https://docs.github.com/en/authentication/keeping-your-account-"
            r"and-data-secure/creating-a-personal-access-token"+"\n"
            )
    %pip install git+https://$GITHUB_TOKEN@github.com/AcousticOdometry/AO
    try:
        import ao
    except ImportError:
        GITHUB_TOKEN = None
        raise
else:
    try:
        import ao
    except ImportError:
        raise ImportError(
            "Acoustic Odometry python extension is not installed. Check "
            r"https://github.com/AcousticOdometry/AO#readme"
            " for detailed instructions."
            )

## Other packages


In [6]:
import ao
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

# Training

## Model definition

In [7]:
class TransformerEncoder(layers.Layer):

    def __init__(self, embed_dim, num_heads, feed_forward_dim, dropout=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
            )
        self.ffn = keras.Sequential([
            layers.Dense(feed_forward_dim, activation="relu"),
            layers.Dense(embed_dim),
            ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout)
        self.dropout2 = layers.Dropout(dropout)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [8]:
class Transformer(keras.Model):
    pass

## Load data

Data folder should be provided as an environment variable. It can be written in
an `.env` file in the root of the project.

In [9]:
if COLAB_RUNTIME:
    # TODO Mount drive and find VAO_Primitive-Experiment
    # TODO Otherwise prompt to add shortcut to drive from link
    # https://drive.google.com/drive/folders/1I6dq8gJpsrD3C14-WQKD4IFOnkiVuvFX
    raise NotImplementedError()
else:
    DATA_FOLDER = ao.dataset.utils.get_data_folder(env='PRIMITIVE_EXPERIMENT')

In [10]:
data, naming = ao.dataset.utils.list_data(DATA_FOLDER)

c:\Users\esdan\Desktop\AO\venv\lib\site-packages\ao\dataset\utils.py:124: UserWarning: Could not parse VAO_2022-03-08_17-11-21 from C:\Users\esdan\OneDrive - keio.jp\Thesis\Primitive Experiment\VAO_2022-03-08_17-11-21, item VAO_2022-03-08_17-11-21 should be composed by one `key` and one `value` separated by a unique underscore but 2 `_` were found.
  warn(str(e))
c:\Users\esdan\Desktop\AO\venv\lib\site-packages\ao\dataset\utils.py:124: UserWarning: Could not parse VAO_2022-03-08_17-12-37 from C:\Users\esdan\OneDrive - keio.jp\Thesis\Primitive Experiment\VAO_2022-03-08_17-12-37, item VAO_2022-03-08_17-12-37 should be composed by one `key` and one `value` separated by a unique underscore but 2 `_` were found.
  warn(str(e))


In [11]:
FRAME_SAMPLES = 1024
NUM_FEATURES = 64

audio_sample, samplerate = ao.io.wave_read(list(data.keys())[0] / 'audio0.wav')
extract = ao.extractor.GammatoneFilterbank(
    FRAME_SAMPLES, NUM_FEATURES, samplerate
    )


def generate_audio_dataset(
        data,
        frame_samples: int = FRAME_SAMPLES,
        num_features: int = NUM_FEATURES
    ):
    for folder, parameters in data:
        audio_path = folder / 'audio0crop.wav'
        audio, _samplerate = ao.io.wave_read(audio_path)
        assert _samplerate == samplerate, "Samplerate mismatch"
        num_frames = tf.math.ceil(audio.size / frame_samples)
        vx = parameters['contact'] * parmeters['w'] * 0.10  # 10 cm radius
        motion = (vx, 0)  # No angular rotation
        # Pad the data to be a multiple of the frame size
        audio = np.append(
            audio, np.zeros(num_frames * frame_samples - audio.size)
            )
        for frame_num in range(num_frames):
            frame = audio[frame_num * frame_samples:(frame_num + 1) *
                          frame_samples]
            frame_features = extract(frame)
            yield (frame_features, motion)


audio_ds = tf.data.Dataset.from_generator(generate_audio_dataset)
print(audio_ds.take(1))

TypeError: To specify the output signature you need to provide either the `output_signature` argument or the `output_types` argument.